In [13]:
import timm
import torch
import torchsummary
from ignite.contrib.handlers import TensorboardLogger, global_step_from_engine
from ignite.engine import (Engine, Events, create_supervised_evaluator,
                           create_supervised_trainer)
from ignite.handlers import ModelCheckpoint
from ignite.metrics import Accuracy, Loss
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.models import resnet18
from torchvision.transforms import Compose, Normalize, Resize, ToTensor, v2

torch.manual_seed(17)

NUM_FINETUNE_CLASSES = 10

model = timm.create_model(
    "vgg19_bn", pretrained=False, checkpoint_path="/Users/tchuang/GitHub/OpenCVDL2023HW2/src/solutions/q4/results/checkpoint/best_model_28_accuracy=0.9398.pt", num_classes=NUM_FINETUNE_CLASSES
)

In [3]:
# Copyright 2023 tc-haung
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
data_transform = Compose(
    [
        ToTensor(),
        Normalize((0.1307,), (0.3081,)),
        Resize((32, 32)),
        v2.Lambda(lambda x: x.repeat(3, 1, 1)),
    ]
)

val_loader = DataLoader(
    MNIST(download=True, root=".", transform=data_transform, train=False),
    batch_size=256,
    shuffle=False,
)

100%|██████████| 9912422/9912422 [00:01<00:00, 6681154.91it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 1301470.77it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2528901.79it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4240046.47it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [14]:
from tqdm import tqdm
correct = 0
count = 0

for batch in tqdm(val_loader):
    images, labels = batch
    with torch.no_grad():
        logits = model(images)
    predicted_label = logits.argmax(-1) 
    matching_count = torch.sum(predicted_label == labels).item()
    correct += matching_count
    count += len(labels)

print(f"Accuracy: {correct / count}")

  0%|          | 0/40 [00:00<?, ?it/s]/Users/tchuang/.local/share/virtualenvs/OpenCVDL2023HW2-jWty1gEm/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 40/40 [01:26<00:00,  2.17s/it]

Accuracy: 0.9485


In [12]:
print(correct, count)
# Copyright 2023 tc-haung
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.



1004 10000


In [ ]:
# Copyright 2023 tc-haung
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

